# Augmenting the dataset with Mirrors

### Random Forest shoots up significantly but KNN (untuned) takes the lead

There clearly is a significant gain from the mirrors on both classifiers but Random Forest is really starting to shine with more data. Which makes me consider XGB for later rounds.

- RF: with more tree (500) and data does very well
- KNN: untuned does very well

Examining both of the results however just confuses me as far as insights go. (the patterns aren't clear)
and KNN's probability precision really isn't as good it seems though as a binary clf it's strong i.e. it's good figuring out if it's a right swipe but bad for examining the deciles

In [1]:
from image_helpers import *
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')


/Users/ajay/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#negs
_, paths, encodings, arrays_rescaled, X = load_encodings('../data/tinder_pics_dislikes_faces_encodings.pickle')
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_dislikes_faces_mirrors_encodings.pickle')
num_of_dislikes = len(paths)

#pos
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_2017_deduped_faces_encodings.pickle')
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_likes_deduped_faces_encodings.pickle')
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_2017_deduped_faces_mirrors_encodings.pickle')
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_likes_deduped_faces_mirrors_encodings.pickle')


num_of_dislikes

4911 4911 4911 4911
5889 5889 5889 5889
6206 6206 6206 6206
7183 7183 7183 7183
7500 7500 7500 7500


4911

In [3]:
tinder_labels = np.r_[np.zeros(num_of_dislikes), np.ones(len(paths) - num_of_dislikes )]
tinder_labels.shape

(7500,)

# All mirrors added

- LR: one point gain
- RF: 8 point gain, 13 (jump from doing a basic n_estimator increase is even more pronounced)

In [19]:
lr = LogisticRegression(class_weight='balanced').fit(encodings, tinder_labels)

score_classifier(lr, encodings, tinder_labels)

(0.8119660437863996,
 0.8068837339365381,
 0.021018287245093376,
 array([0.80619644, 0.7958622 , 0.80757103, 0.85538142, 0.79445462,
        0.8047716 , 0.82634919, 0.77826357, 0.8343071 , 0.81650326]))

In [20]:
joblib.dump(lr, 'lr_tinderlikes3.joblib')

['lr_tinderlikes3.joblib']

# The following indicates tuning would help

In [12]:
rf = RandomForestClassifier(class_weight='balanced_subsample').fit(encodings, tinder_labels)

score_classifier(rf, encodings, tinder_labels)

(0.87113677692162,
 0.8748938420527016,
 0.00873054660137848,
 array([0.85987659, 0.87875485, 0.87489384, 0.86153563, 0.88062297]))

In [16]:
rf = RandomForestClassifier(class_weight='balanced_subsample', n_estimators=50, n_jobs=-1).fit(encodings, tinder_labels)

score_classifier(rf, encodings, tinder_labels)

(0.9261479887772197,
 0.9274174523665358,
 0.0053827713197829325,
 array([0.92757083, 0.93435999, 0.92741745, 0.91801756, 0.92337412]))

In [17]:
rf = RandomForestClassifier(class_weight='balanced_subsample', n_estimators=500, n_jobs=-1).fit(encodings, tinder_labels)

score_classifier(rf, encodings, tinder_labels)

(0.9404783313400715,
 0.9445501655277623,
 0.005365093217454689,
 array([0.93253357, 0.94498364, 0.93548644, 0.94455017, 0.94483784]))

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(metric='cosine').fit(encodings, tinder_labels)

score_classifier(knn, encodings, tinder_labels)

(0.9521045875201184,
 0.9527085943667836,
 0.005650789775609924,
 array([0.95270859, 0.94257641, 0.958391  , 0.94985217, 0.95699476]))

# Mirrors of ONLY the positive class added

In [6]:
lr = LogisticRegression(class_weight='balanced').fit(encodings, tinder_labels)

score_classifier(lr, encodings, tinder_labels)

(0.8125806560198432,
 0.8172462795563099,
 0.007972706068891812,
 array([0.82111993, 0.800327  , 0.80608305, 0.81812702, 0.81724628]))

In [7]:
rf = RandomForestClassifier(class_weight='balanced_subsample').fit(encodings, tinder_labels)

score_classifier(rf, encodings, tinder_labels)

(0.8464743268291416,
 0.8469742337089275,
 0.003001983747369949,
 array([0.84413065, 0.84697423, 0.85120755, 0.84755338, 0.84250582]))

In [19]:
rf

RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [20]:
lr

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [27]:
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

# Grid Search

- next time try random cv to minimize the grid

```
from sklearn.model_selection import RandomizedSearchCV
rf_params = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
 
 # Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)
 ```
 
 


In [61]:
knn_cv.best_estimator_.predict_proba(encodings)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [9]:
from modeling_helpers import *

rf_params = {
    'bootstrap':[True, False],  
    'max_depth': [None, 80, 90, 100, 110],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000, 1500],
    'class_weight':['balanced_subsample'],
    'n_jobs':[-1]
}


rf_cv = random_grid_searching_model(RandomForestClassifier(), np.array(encodings), tinder_labels,params=rf_params, folds=10)
rf_cv.best_params_, rf_cv.best_score_


Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True 
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True 
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True 
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True, score=0.9277788241202877, total= 1.6min
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=3, max_depth=None, class_weight=balanced_subsample, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=1000, min_samples_spli

[CV] n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True 


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 10.8min


[CV]  n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True, score=0.8875118936218732, total= 2.4min
[CV] n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True, score=0.9340483922968649, total= 2.4min
[CV] n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=1500, min_samples_split=12, min_samples_leaf=5, max_depth=80, class_weight=balanced_subsample, bootstrap=True, score=0.9257287546493249, total= 2.5min
[CV] n_jobs=-1, n_estimators=1000, min_samples_split=8, min_samples_leaf=5, max_depth=110, class_weight=balanced_subsample, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=1500, m

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 19.7min finished



 Time taken: 0 hours 19 minutes and 48.65 seconds.
None

 Best estimator:
RandomForestClassifier(bootstrap=False, class_weight='balanced_subsample',
            criterion='gini', max_depth=90, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=12, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

 Best score:
0.9456979735192265

 Best hyperparameters:
{'n_jobs': -1, 'n_estimators': 200, 'min_samples_split': 12, 'min_samples_leaf': 3, 'max_depth': 90, 'class_weight': 'balanced_subsample', 'bootstrap': False}


({'bootstrap': False,
  'class_weight': 'balanced_subsample',
  'max_depth': 90,
  'min_samples_leaf': 3,
  'min_samples_split': 12,
  'n_estimators': 200,
  'n_jobs': -1},
 0.9456979735192265)

In [12]:

partial_param_grid = {
    'bootstrap':[False],  
    'max_depth': [50, 90],
    'min_samples_leaf': [1,3, 4],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 800, 1200],
    'class_weight':['balanced_subsample'],
    'n_jobs':[-1]
}

results = full_gridsearch(RandomForestClassifier(), np.array(encodings), tinder_labels,param_grid=partial_param_grid, folds=10)
results

Best: 0.959720 using {'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 90, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 1200, 'n_jobs': -1}
0.957315 (0.007491) with: {'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 200, 'n_jobs': -1}
0.959671 (0.007241) with: {'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 800, 'n_jobs': -1}
0.959497 (0.006821) with: {'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 1200, 'n_jobs': -1}
0.955330 (0.006544) with: {'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200, 'n_jobs': -1}
0.957468 (0.007805) with: {'bootstrap': False, 'class_weight': 'balanced_subsample

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1a174ffeb8>,
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [False], 'max_depth': [50, 90], 'min_samples_leaf': [1, 3, 4], 'min_samples_split': [8, 10, 12], 'n_estimators': [200, 800, 1200], 'class_weight': ['balanced_subsample'], 'n_jobs': [-1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [13]:
results.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight='balanced_subsample',
            criterion='gini', max_depth=90, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=8, min_weight_fraction_leaf=0.0,
            n_estimators=1200, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [14]:
results.best_score_

0.9597200903134085

In [15]:
import joblib
joblib.dump(results.best_estimator_, 'rf_tinderlikes3.joblib')

['rf_tinderlikes3.joblib']

In [57]:
from sklearn.model_selection import GridSearchCV


# for 'mahalanobis' look-up

# nn = NearestNeighbors(algorithm='brute', 
#                    metric='mahalanobis', 
#                    metric_params={'V': np.cov(X)})

knn_params = {
    'n_neighbors':list(range(1,50,10)),
    'metric':['hamming','correlation','minkowski', 'euclidean', 'chebyshev' ],
    'weights':['distance', 'uniform'],
}

knn_cv = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid=knn_params, 
    n_jobs=-1,
    cv=10, 
    scoring='roc_auc'
).fit(encodings, tinder_labels)

#({'metric': 'correlation', 'n_neighbors': 1, 'weights': 'distance'}, 0.9931348072641333)


knn_cv.best_params_, knn_cv.best_score_

({'metric': 'correlation', 'n_neighbors': 1, 'weights': 'distance'},
 0.9931348072641333)

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn_best_params = {'metric': 'correlation', 'n_neighbors': 1, 'weights': 'distance'}
knn = KNeighborsClassifier(**knn_best_params).fit(encodings, tinder_labels)

score_classifier(knn, encodings, tinder_labels)

(0.9791225786102078,
 0.9797611839363367,
 0.004266001545918375,
 array([0.98242145, 0.97833985, 0.97367283, 0.97184849, 0.98321918,
        0.97935818, 0.98638033, 0.98016419, 0.98058882, 0.97523248]))

In [18]:

joblib.dump(knn, 'knn_tinderlikes3.joblib')


['knn_tinderlikes3.joblib']